In this Jupyter notebook, you'll explore the PageRank algorithm and how changing the topology of a network of webpages can affect the PageRank of individual pages. 

First, let's create a simple data abstraction to represent a webpage.

In [243]:
class Webpage:    
    # fixed_pagerank is used to model an external page whose PR should not be updated
    # The default, -1, means that the page's PR should be updated.
    def __init__(self, name, fixed_pagerank=-1):
        # create a new webpage, with no links
        self.name = name
        self.links = []
        self.backlinks = []
        self.fixed_pagerank = fixed_pagerank

    def add_link(self, target_page):
        if not target_page in self.links:
            self.links.append(target_page)
        # when adding a link, add a backlink on the target
        target_page.add_back_link(self) 
        
    def add_back_link(self, source_page):
        if not source_page in self.backlinks:
            self.backlinks.append(source_page)

    # debugging methods        
    def print_links(self):
        for page in self.links:
            print (page.name + ", ")
    def print_back_links(self):
        for page in self.backlinks:
            print (page.name + ", ")
    def __str__(self):
        return self.name        

We provide a simple iterative implementation of PageRank:

In [244]:
class PageRank:
    def __init__(self, pages, damping_factor=0.85, include_supernode=True, debug=False):
        self.page_rank_table = {}
        self.damping_factor = damping_factor
        self.debug=debug

        # create a "supernode" that has a link and backlink to every page
        self.pages = pages.copy() # don't update the actual pages
        
        if include_supernode == True:
        
            supernode = Webpage("supernode")
        
            for page in self.pages:
                if page.fixed_pagerank == -1:
                    page.add_link(supernode)
                    supernode.add_link(page)

            self.pages.append(supernode)
        
        for page in self.pages:
            # initialize each page's PR to be 1/n, where n is the total number of pages
            self.page_rank_table[page] = 1/len(self.pages)
          
    def run_page_rank(self, iterations):
        for ii in range(iterations):
            if self.debug:
                print("\nIteration #" + str(ii))
                self.print_table(show_supernode=True)
                
            new_page_rank_table = {}
            for page in self.page_rank_table:
                if page.fixed_pagerank == -1:
                    new_page_rank = 0
                    for backlink in page.backlinks:
                        new_page_rank += self.page_rank_table[backlink] / len(backlink.links)
                    new_page_rank_table[page] = (1-self.damping_factor) + self.damping_factor * new_page_rank
                else:
                    new_page_rank_table[page] = page.fixed_pagerank
            self.page_rank_table = new_page_rank_table

    # debugging & validation methods
    def calc_average_pagerank(self):
        sum_pagerank = 0
        for page in self.page_rank_table:
            sum_pagerank += self.page_rank_table[page]
        return sum_pagerank / len(self.page_rank_table)

    def print_table(self, show_supernode=False):
        for page in self.page_rank_table:
            if str(page) != "supernode" or show_supernode:
                print(str(page) + ": " + str(self.page_rank_table[page]))        

Create a simple network, consisting of two pages, each linking to each other, and calculate the PageRank of each one.

In [245]:
pages = []
pageA = Webpage("A")
pageB = Webpage("B")

pageA.add_link(pageB)
pageB.add_link(pageA)

pages.append(pageA)
pages.append(pageB)

pageRank = PageRank(pages)
pageRank.run_page_rank(1000)
pageRank.print_table(show_supernode=True)

A: 0.9999999999999997
B: 0.9999999999999997
supernode: 0.9999999999999997


# Assignment Exercises
Complete all three of the following exercises.

### Exercise 1
While the inputs to Google's overall ranking algorithm are secret, the output of PageRank is not. There are tools that  allow you to enter a URL and see the Google PageRank of the corresponding page. For example, if you enter nuevaschool.org into [this one](https://www.prchecker.info/check_page_rank.php), it will tell you that Nueva's homepage has a PR of 4.

Two notes:
1. Recall that this is not the raw output -- the PageRank algorithm can produce an arbitrarily high output, so to make it easier to understand, Google maps the output to a scale of 0-10, where 0 is low and 10 is high.
2. This tool is kind of a pain to use since it requires that you enter a captcha every time you submit. If you find a better one, let me know! Just be careful, as many of these tools have a bunch of spammy ads on them.

#### Questions
1. Play around with this tool and report back the values of some webpages. Can you find webpages of each rank between 0 and 10? 
2. Describe what kinds of webpages are represented in each rank. For example, you might say that pages of rank 9 are usually the homepages of the services of the biggest Internet companies in the world.

**Question 1:**

0: noot.space, many many others

4: nuevaschool.org

5: bestdelegate.com

6: artofproblemsolving.com

7: duolingo.com

8: reddit.com

9: google.com

10: usa.gov

I spent a long time trying and failing to find any websites with pagerank 1-3 :/

**Question 2:**

There are less than 10 webpages total that have pagerank 10. These are the websites that the algorith believes are most authentic or reliable, such as the homepage of the US government. Pages of rank 9 include most of the most popular websites, such as Google, Facebook, Wikipedia, etc. Ranks 5-8 seem to contain recognizeable (not sketchy) sites that people across the country/world might be interested in, with its nicheness/specifity increasing as pagerank decreases. PR 4 seems to include web pages of specific institutions/organizations that only people who are affiliated with them might be interested in, such as private schools and events. I couldn't find any pages with ranks 1-3, and PR 0 contains websites that are extremely irrelevant or sketchy (such as the one I listed). However, more legitimate pages such as the San Mateo County website (smcgov.org) are also ranked 0, so maybe there's an issue with the prchecker website.

### Exercise 2
In the given implementation, we set the damping factor to 0.85 and we initialize each page's PageRank to be 1/n, where n is the total number of pages. 

1. What happens if you change the damping factor? How does the algorithm behave with a small damping factor vs a large one? 
2. What happens if you change the initial PageRank?

You may want to use the `debug=True` flag and the `calc_average_pagerank` method to get a better look at what's going on.

In [246]:
pages = []
pageA = Webpage("A")
pageB = Webpage("B")
pageC = Webpage("C")
pageD = Webpage("D")

pageA.add_link(pageB)
pageA.add_link(pageC)
pageB.add_link(pageC)
pageC.add_link(pageA)
pageD.add_link(pageC)
# Pages set up the same way as the example in the lecture slides

pages.append(pageA)
pages.append(pageB)
pages.append(pageC)
pages.append(pageD)

pageRank = PageRank(pages, damping_factor=0.3, include_supernode=False)
# I changed the PageRank class to have an option whether or not to have a supernode

pageRank.page_rank_table[pageA] = 100
pageRank.page_rank_table[pageB] = -33
pageRank.page_rank_table[pageC] = 0.25
pageRank.page_rank_table[pageD] = 0

pageRank.run_page_rank(50)
pageRank.print_table(show_supernode=True)
pageRank.calc_average_pagerank()

A: 1.100371747211896
B: 0.8650557620817844
C: 1.3345724907063197
D: 0.7


1.0

**Changing damping factor**

The damping factor controls how extreme each page's PR is. When the damping factor is small, all pages have a very similar pagerank; when the damping factor is large, their PRs vary a lot. This is because, in the pagerank formula, the damping factor controls how heavily the part that is different for each page is weighted. When the damping factor is small, such as 0.2, the sum of the PR(i)/C(i) quotients, which is the part that is different for each page, is only 20% of the score (while all pages have the same large constant, 0.8), so it's difficult to make the scores that different. The converse is true with a large damping constant, where the sum of PR(i)/C(i) quotients play a much larger role (and the constant a much smaller role) in determining a page's score. However, changing the damping constant will *not* change the *rank* of each page, only its actual PR *value*.

When the damping factor is larger than 1 or less than -1, the pagerank of each page does not converge, instead decreasing to negative infinity. This is because either d or 1-d becomes negative, decreasing the PR forever. A pages PR does still converge to a fixed number if the damping factor is between -1 and 0, although it might be negative and the order will be different. However we can disregard all these cases because they represent a negative probability which (I'm pretty sure) doesn't exist.


**Changing initial PR**

Changing the initial PR of a page does not affect its final PR after a large number of iterations. The initial PRs can be positive, negative, zero, decimals, very large, or very small but they will eventually all converge to the same values as if they all started with 1/n (as long as the damping factor represents a valid probability less than 1). And no matter what the initial PR values are for the pages, the final average pagerank after many iterations is always 1. However, if the initial PRs are very different, it would take more iterations for the PRs to converge to their final stable values.

### Exercise 3
In class, we discussed the major advantage of PageRank compared to the search engine algorithms that came before it: since PageRank is determined by backlinks, which the owner of a page has less control over, it's highly resistant to *keyword stuffing*, which is the practice of putting a ton of keywords on your webpage so that the indexer thinks your page is relevant to all those keywords. In this exercise, we'll explore what kinds of practices can manipulate PageRank; this is known as search engine optimization, or SEO, and is a $80 billion industry in the US alone.


1. You've just created a startup, and you need to make a website for your startup. You have four pages to start with:
    1. Homepage
    2. About (the story of the company)
    3. Team (the bios of you and your cofounders)
    4. Blog
    
  How should you structure your website so that your homepage has the highest possible PageRank? List the PageRank values for each of the four webpages in this structure.
  
  
2. Your product is a hit! Glowing reviews are rolling in, and you want to add a page for Reviews that lists and links all of the reviews (5 so far). How does this affect your homepage's PageRank? Should you restructure your website, and if so, how? List the PageRank values for each of the five webpages you control (Home, About, Team, Blog, and Reviews).


3. It's been 3 months, and you've produced a blog post every week (total of 12 posts). Your homepage still has a link to the most recent post, and each post links to the post before it and after it, like so:

  `Home ==> Blog post 12 <==> Blog post 11 <==> Blog post 10 <==> ... <== Blog post 1`
  
 How does this affect your homepage's PageRank? Should you restructure your website, and if so, how? Do websites you visit structure their sites in this way? If not, what are some reasons why?
 
4. Copycats of your product are popping up all over the web, and your homepage is getting pushed off the first page of search results. You decide you need to take drastic action. What are some unscrupulous ways you might boost the PageRank of your homepage quickly? Model each of these and give their impact on the PageRank of your homepage.

5. Google figures out what you're doing, and they send you a nice cease and desist letter telling you to stop, or they'll remove your website from the search results entirely. What are some legitimate ways you might boost the PageRank of your homepage?


**Part 1**

In [387]:
pages = []
home = Webpage("Home")
about = Webpage("About")
team = Webpage("Team")
blog = Webpage("Blog")

home.add_link(about)
home.add_link(team)
home.add_link(blog)
about.add_link(home)
team.add_link(home)
blog.add_link(home)

pages.append(home)
pages.append(about)
pages.append(team)
pages.append(blog)

PR = PageRank(pages)

PR.run_page_rank(1000)
PR.print_table(show_supernode=True)

Home: 1.3893129770992356
About: 0.7404580152671751
Team: 0.7404580152671751
Blog: 0.7404580152671751
supernode: 1.3893129770992356


The configuration in which the homepage has the highest possible pagerank is when home has links to all other pages and about, team, and blog each only have links to home. This gives the homepage a PR of 1.389 and all other pages a PR of 0.7405. In this scenario, the homepage kind of acts as a second supernode (actually this configuration doesn't even need a supernode but I am including it to be consistent with the next parts).

**Part 2**

In [386]:
# I'm going to copy all the previous code in each part
# in order to avoid linking to multiple supernodes or other fun problems :)

pages = []
home = Webpage("Home")
about = Webpage("About")
team = Webpage("Team")
blog = Webpage("Blog")
reviews = Webpage("Reviews")

home.add_link(about)
home.add_link(team)
home.add_link(blog)
home.add_link(reviews)

about.add_link(home)
team.add_link(home)
blog.add_link(home)
reviews.add_link(home)

pages.append(home)
pages.append(about)
pages.append(team)
pages.append(blog)
pages.append(reviews)

r1 = Webpage("Review #1")
r2 = Webpage("Review #2")
r3 = Webpage("Review #3")
r4 = Webpage("Review #4")
r5 = Webpage("Review #5")

reviews.add_link(r1)
reviews.add_link(r2)
reviews.add_link(r3)
reviews.add_link(r4)
reviews.add_link(r5)

pages.append(r1)
pages.append(r2)
pages.append(r3)
pages.append(r4)
pages.append(r5)

# I have to include the reviews in the system in order to make sure the average PR is still 1.0
# but then the reviews also get pageranked and that might mess up the other ones??? eee

PR = PageRank(pages)
PR.run_page_rank(1000)
PR.print_table(show_supernode=True)
PR.calc_average_pagerank()

Home: 1.4869821279672464
About: 0.7259841208972726
Team: 0.7259841208972726
Blog: 0.7259841208972726
Reviews: 0.7259841208972726
Review #1: 0.5613523738232239
Review #2: 0.5613523738232239
Review #3: 0.5613523738232239
Review #4: 0.5613523738232239
Review #5: 0.5613523738232239
supernode: 3.8023195193275385


0.9999999999999997

One way to keep the home at a high pagerank is to have the reviews page link to home only and not have any page link to reviews (in addition to the configuration described in part 1). This way home would have a PR of 1.568, about, team, and blog has 0.8104, and reviews has a PR of 0.4771.

However, that way would be unrealistic for an actual website in that there is no way to access the reviews page. If the homepage also linked to the reviews page, the home pagerank would drop to 1.487, but that is still the highest score I could get in comparison to linking to reviews from about or team etc. In this case, about, team, blog, and reviews all have PRs of 0.7260.

**Part 3**

In [434]:
pages = []
home = Webpage("Home")
about = Webpage("About")
team = Webpage("Team")
blog12 = Webpage("Blog 12")
reviews = Webpage("Reviews")

home.add_link(about)
home.add_link(team)
home.add_link(blog12)
home.add_link(reviews)

about.add_link(home)
team.add_link(home)
blog12.add_link(home)
reviews.add_link(home)

pages.append(home)
pages.append(about)
pages.append(team)
pages.append(blog12)
pages.append(reviews)

r1 = Webpage("Review #1")
r2 = Webpage("Review #2")
r3 = Webpage("Review #3")
r4 = Webpage("Review #4")
r5 = Webpage("Review #5")

reviews.add_link(r1)
reviews.add_link(r2)
reviews.add_link(r3)
reviews.add_link(r4)
reviews.add_link(r5)

pages.append(r1)
pages.append(r2)
pages.append(r3)
pages.append(r4)
pages.append(r5)

blog_pages = [None]*11
for ii in range(11):
    blog_pages[ii] = Webpage("Blog " + str(ii+1))
    pages.append(blog_pages[ii])
blog_pages[0].add_link(blog_pages[1])
blog_pages[0].add_link(home)
for ii in range(1,10):
    blog_pages[ii].add_link(blog_pages[ii-1])
    blog_pages[ii].add_link(blog_pages[ii+1])
    blog_pages[ii].add_link(home)
blog_pages[10].add_link(blog_pages[9])
blog_pages[10].add_link(blog12)
blog_pages[10].add_link(home)
blog12.add_link(blog_pages[10])

In [391]:
PR = PageRank(pages)
PR.run_page_rank(1000)
PR.print_table(show_supernode=True)
PR.calc_average_pagerank()

Home: 3.122005126838981
About: 0.902402706487509
Team: 0.902402706487509
Blog 12: 1.0771997859424842
Reviews: 0.902402706487509
Review #1: 0.48123930642693685
Review #2: 0.48123930642693685
Review #3: 0.48123930642693685
Review #4: 0.48123930642693685
Review #5: 0.48123930642693685
Blog 1: 0.510650536471428
Blog 2: 0.6540644778660977
Blog 3: 0.6480862318791099
Blog 4: 0.646756213683209
Blog 5: 0.6464755505130161
Blog 6: 0.6464848008491859
Blog 7: 0.6468089950131193
Blog 8: 0.6483253642719293
Blog 9: 0.6551370254435732
Blog 10: 0.6856755322866169
Blog 11: 0.8225744915528255
supernode: 5.476351215791207


0.9999999999999997

When the blog posts are all linked and added to the pagerank system, the homepage's PR drops from 1.487 to 1.220. Interestingly, the pages with the next highest pageranks are all the blog posts, with PRs between 0.91 and 0.65.

To increase the PR of my homepage, I can link all the blog posts to home. That significantly boosts the homepage's PR to 3.122, much higher than before. At the same time, it lowers the PR of all blog posts 1-11 to around 0.65, allowing the next most important pages (About, Team, Blog, and Reviews), to each have higher PRs of 0.9024.

**Part 4**

The first way to artificially boost the homepage's PR is to increase the damping factor. As mentioned in question 1, the higher the damping factor, the more extreme each page's PR becomes. And since the homepage's PR is already high, increasing the damping factor will make it even higher.

In [405]:
# make sure to run the first part of the code in part 3 (without running the second half) before running this
# every time

PR = PageRank(pages, damping_factor=0.99)
PR.run_page_rank(1000)
PR.print_table(show_supernode=True)
PR.calc_average_pagerank()

Home: 3.4085789385425773
About: 0.9633935509864109
Team: 0.9633935509864109
Blog 12: 1.1695604083303666
Reviews: 0.9633935509864109
Review #1: 0.42474652037308674
Review #2: 0.42474652037308674
Review #3: 0.42474652037308674
Review #4: 0.42474652037308674
Review #5: 0.42474652037308674
Blog 1: 0.43002351766240204
Blog 2: 0.5718318067412769
Blog 3: 0.5714301360531947
Blog 4: 0.5713401076660826
Blog 5: 0.5713780273276318
Blog 6: 0.5716212665898319
Blog 7: 0.5725661322789717
Blog 8: 0.5761405338550993
Blog 9: 0.5896377010444154
Blog 10: 0.6405973318566917
Blog 11: 0.8329977507612835
supernode: 5.90747649026987


0.9999587910820167

By increasing the damping factor from 0.85, which is the default, to 0.99, the homepage's PR increases from 3.22 to 3.41.

When I increase the damping factor even more to 0.999, the average PR goes down to around 0.6 for some reason (why?), so I would keep it at 0.99.

Another way to boost the homepage's PR would be to create a bunch of webpages that only link to homepage but don't contain any content. The homepage should also link to those fake pages so that the fake pages get higher PR, which then boost home's PR.

In [435]:
fake_pages = [None] * 1000
for ii in range(1000):
    fake_pages[ii] = Webpage("Fake" + str(ii))
    fake_pages[ii].add_link(home)
    home.add_link(fake_pages[ii])
    pages.append(fake_pages[ii])

In [436]:
# make sure to run the first part of the code in part 3 (without running the second half)
# and then run the box above this
# every time

PR = PageRank(pages, damping_factor=0.99)
PR.run_page_rank(1000)
PR.print_table(show_supernode=True)
PR.calc_average_pagerank()

Home: 252.69710315541957
About: 0.5055269224124699
Team: 0.5055269224124699
Blog 12: 0.6552496832157497
Reviews: 0.5055269224124699
Review #1: 0.32809744912050365
Review #2: 0.32809744912050365
Review #3: 0.32809744912050365
Review #4: 0.32809744912050365
Review #5: 0.32809744912050365
Blog 1: 0.38248340450144264
Blog 2: 0.5086138552150421
Blog 3: 0.5082541804301681
Blog 4: 0.5081649679088787
Blog 5: 0.5081641876782441
Blog 6: 0.5082502479039515
Blog 7: 0.5085987463725937
Blog 8: 0.5099207613961589
Blog 9: 0.514913740317417
Blog 10: 0.5337653861512172
Blog 11: 0.6049407087183719
Fake0: 0.5055269224124699
Fake1: 0.5055269224124699
Fake2: 0.5055269224124699
Fake3: 0.5055269224124699
Fake4: 0.5055269224124699
Fake5: 0.5055269224124699
Fake6: 0.5055269224124699
Fake7: 0.5055269224124699
Fake8: 0.5055269224124699
Fake9: 0.5055269224124699
Fake10: 0.5055269224124699
Fake11: 0.5055269224124699
Fake12: 0.5055269224124699
Fake13: 0.5055269224124699
Fake14: 0.5055269224124699
Fake15: 0.505526922

0.9999568709945406

When 1000 fake pages are linked to and from home, home's PR increases from 3.41 to 253, which is just 1 point smaller than the supernode. This is because the large amount of fake pages makes the pages that don't link to home (the five reviews) negligible. In this case, the fake pages have the same PR as the other useful pages (About, Team, etc), but I guess we don't care about that :P

The more fake pages added, the higher home's PR becomes. However it also makes calculating PR (and your website in general probably) much slower, so don't use values that are too large.

Finally, you can always get a job at Google and manipulate the pagerank algorithm so that it puts your page as the top result for any search. But you also will get fired.

**Part 5**

Some more legitimate ways to increase your pagerank might include:
1. *Asking* other sites with high PR to link to yours. For example, you can ask people who leave reviews to link back to your homepage (as is logical)
2. Linking your page to your social media (which has an extremely high pagerank) so people can access it from there
3. Basically any other way to advertise/promote your website legally, such as forums etc.
4. ummmm
5. Actually create an extremely popular website that people want to visit???